In [80]:
# Imports
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [74]:
# input.csvの読み込み
input_df = pd.read_csv("input.csv")

In [75]:
input_df

,地点,完全住所
0,A,熊本市東区榎町15-154
1,B,愛知県豊明市沓掛町山新田２丁目４０−８


In [76]:
# 完全住所から住所を抽出し、緯度・経度をタプルで取得し新たな列「緯度・経度」を作成
# ジオコーダーの初期化
geolocator = Nominatim(user_agent="geoapi")
# 新規列を事前に作成
input_df['住所'] = None
input_df['緯度・経度'] = None
for index, row in input_df.iterrows():
    match = re.search(r'\d', row['完全住所'])
    if match:
        address = row['完全住所'][:match.start()]
    else:
        address = row['完全住所']
    input_df.at[index, '住所'] = address
    # 緯度・経度をタプルで取得し新たな列「緯度・経度」を作成
    location = geolocator.geocode(address)
    if location:
        input_df.at[index, '緯度・経度'] = (location.latitude, location.longitude)
    else:
        input_df.at[index, '緯度・経度'] = None

In [78]:
# temple_listの読み込み
temple_list_df = pd.read_csv("temple_list.csv")

In [79]:
# 緯度と経度をタプルにして新しい列を作成し、元の列を削除
temple_list_df['緯度・経度'] = list(zip(temple_list_df['緯度'], temple_list_df['経度']))
temple_list_df = temple_list_df.drop(['緯度', '経度'], axis=1)

In [81]:
# 各入力地点に対して最も近い寺院を見つける関数
def find_nearest_temple(input_coords, temple_df):
    """入力座標に最も近い寺院を見つける"""
    if input_coords is None:
        return None, None
    
    min_distance = float('inf')
    nearest_temple_idx = None
    
    for idx, temple_coords in enumerate(temple_df['緯度・経度']):
        if temple_coords:
            distance = geodesic(input_coords, temple_coords).km
            if distance < min_distance:
                min_distance = distance
                nearest_temple_idx = idx
    
    if nearest_temple_idx is not None:
        return temple_df.iloc[nearest_temple_idx], min_distance
    return None, None

# 最寄りの寺院情報を格納するリスト
nearest_temple_data = []

# 各入力地点に対して最寄りの寺院を見つける
for index, row in input_df.iterrows():
    nearest_temple, distance = find_nearest_temple(row['緯度・経度'], temple_list_df)
    
    if nearest_temple is not None:
        temple_info = {
            '最寄り寺院名': nearest_temple['寺院名'],
            '最寄り寺院_完全住所': nearest_temple['完全住所'],
            '最寄り寺院_住所': nearest_temple['住所'],
            '最寄り寺院_緯度・経度': nearest_temple['緯度・経度'],
            '距離(km)': round(distance, 2)
        }
    else:
        temple_info = {
            '最寄り寺院名': None,
            '最寄り寺院_完全住所': None,
            '最寄り寺院_住所': None,
            '最寄り寺院_緯度・経度': None,
            '距離(km)': None
        }
    
    nearest_temple_data.append(temple_info)

# データフレームに変換
nearest_temple_df = pd.DataFrame(nearest_temple_data)

# input_dfと横結合
result_df = pd.concat([input_df, nearest_temple_df], axis=1)

# 結果を表示
result_df

,地点,完全住所,住所,緯度・経度,最寄り寺院名,最寄り寺院_完全住所,最寄り寺院_住所,最寄り寺院_緯度・経度,距離(km)
0,A,熊本市東区榎町15-154,熊本市東区榎町,"(32.7937199, 130.7727527)",本妙寺,熊本県熊本市西区花園4-13-1,熊本県熊本市西区花園,"(32.8144574, 130.7015092)",7.06
1,B,愛知県豊明市沓掛町山新田２丁目４０−８,愛知県豊明市沓掛町山新田,"(35.0914713, 137.0174099)",浄久寺,愛知県名古屋市天白区八事山143（旧住所：八事山144）,愛知県名古屋市天白区八事山,"(35.1363292, 136.9665965)",6.80


In [82]:
# 結果をresult.csvに出力
result_df.to_csv("result.csv", index=False)